In [ ]:
hparams = {
    # --- Model ---
    # | Model hyperparameters
    'num_classes': 3,
    # | Optimization hyperparameters
    "learning_rate": 0.0625,
    "weight_decay": 0.000000625,
    "lr_decay": 0.25,
    
    # --- Dataloader ---
    "batch_size": 12,
    "num_workers": 2,
}